In [1]:
import json, os, requests
import sasoptpy as so
import pandas as pd

DATA_DIR = os.path.join('..', 'data', 'fplreview')

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
fpl_data = requests.get(base_url+'bootstrap-static/').json()

In [3]:
# load all players' projections
df = pd.read_csv(os.path.join(DATA_DIR, 'gw37.csv'))
df.columns = ['Pos', 'Name', 'BV', 'SV', 'Team', 'xMins', 'Pts']

# show top 5 projections
df.sort_values('Pts', ascending=False).head()

,Pos,Name,BV,SV,Team,xMins,Pts
253,M,Salah,12.8,12.7,Liverpool,73,6.771
387,F,Kane,11.8,11.8,Spurs,84,6.253
301,M,Fernandes,11.4,10.9,Man Utd,79,6.236
250,M,Mané,11.7,11.7,Liverpool,77,6.039
258,D,Alexander-Arnold,7.7,7.5,Liverpool,84,4.944


In [4]:
# positions
positions = pd.DataFrame(fpl_data['element_types'])
positions['Pos'] = positions['singular_name'].apply(lambda x: x[0])
positions.set_index('Pos', inplace=True)
positions = positions[['squad_select', 'squad_min_play' ,'squad_max_play']]

In [5]:
positions

,squad_select,squad_min_play,squad_max_play
Pos,,,
G,2,1,1
D,5,3,5
M,5,2,5
F,3,1,3


In [6]:
players = df.index.tolist()
position_list = positions.index.tolist()
team_list = df['Team'].unique().tolist()

In [7]:
model = so.Model(name='single_period')

NOTE: Initialized model single_period.


## Variables

In [8]:
squad = model.add_variables(players, name='squad', vartype=so.binary)
lineup = model.add_variables(players, name='lineup', vartype=so.binary)
captain = model.add_variables(players, name='captain', vartype=so.binary)
vicecap = model.add_variables(players, name='vicecap', vartype=so.binary)

## Constraints

In [9]:
# 15 players in squad
squad_count = so.expr_sum(squad[p] for p in players)
model.add_constraint(squad_count == 15, name='squad_count');

# 11 players in starting lineup
model.add_constraint(
    so.expr_sum(lineup[p] for p in players) == 11, name='lineup_count'
)

# 1 captain
model.add_constraint(
    so.expr_sum(captain[p] for p in players) == 1, name='captain_count'
)

# 1 vice-captain
model.add_constraint(
    so.expr_sum(vicecap[p] for p in players) == 1, name='vicecap_count'
)

# players in starting lineup must also be in squad
model.add_constraints(
    (lineup[p] <= squad[p] for p in players), name='lineup_squad_rel'
)

# captain must come from within squad
model.add_constraints(
    (captain[p] <= lineup[p] for p in players), name='captain_lineup_rel'
)

# vice-captain must come from within squad
model.add_constraints(
    (vicecap[p] <= lineup[p] for p in players), name='vicecap_lineup_rel'
)

# captain and vice-captain can't be same person
model.add_constraints(
    (captain[p] + vicecap[p] <= 1 for p in players), name='cap_vc_rel'
);

In [10]:
# count of each player per position in starting lineup
lineup_type_count = {
    t: so.expr_sum(
        lineup[p] for p in players if df.loc[p, 'Pos'] == t
    ) for t in position_list
}

# count of all players in squad must be at least 'squad_min_play'
# and no more than 'squad_max_play' for each position type 
model.add_constraints(
    (lineup_type_count[t] == [
        positions.loc[t, 'squad_min_play'],
        positions.loc[t, 'squad_max_play']
    ] for t in position_list),
    name='valid_formation'
);

In [11]:
# count of each player per position in squad
squad_type_count = {
    t: so.expr_sum(
        squad[p] for p in players if df.loc[p, 'Pos'] == t
    ) for t in position_list
}

# count of all players in squad must be equal to 'squad_select'
# for each position type 
model.add_constraints(
    (squad_type_count[t] == positions.loc[t, 'squad_select'] for t in position_list),
    name='valid_squad'
);

In [12]:
# total value of squad cannot exceed 100
budget = 100
price = so.expr_sum(df.loc[p, 'BV'] * squad[p] for p in players)
model.add_constraint(price <= budget, name='budget_limit');

In [13]:
# no more than 3 players per team
model.add_constraints(
    (so.expr_sum(
        squad[p] for p in players if df.loc[p, 'Team'] == t
    ) <= 3 for t in team_list),
    name='team_limit'
);

## Optimisation function

In [14]:
# sum of starting 11 players, plus double captain score and upweight vice-captain
total_points = so.expr_sum(
    df.loc[p, 'Pts'] * (lineup[p] + captain[p] + 0.1 * vicecap[p]) for p in players
)

model.set_objective(-total_points, sense='N', name='total_xp');

In [15]:
model.export_mps('single_period.mps')

In [16]:
command = 'cbc single_period.mps solve solu solution_sp.txt'
!{command}

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Feb 26 2021 

command line - cbc single_period.mps solve solu solution_sp.txt (default strategy 1)
At line 1 NAME     single_period
At line 2 ROWS
At line 2705 COLUMNS
At line 9376 RHS
At line 9727 RANGES
At line 9731 BOUNDS
At line 12400 ENDATA
Problem single_period has 2701 rows, 2668 columns and 10672 elements
Coin0008I single_period read with 0 errors
Continuous objective value is -62.419 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2001 strengthened rows, 0 substitutions
Cgl0004I processed model has 2033 rows, 2668 columns (2668 integer (2668 of which binary)) and 10669 elements
Coin3009W Conflict graph built in 0.009 seconds, density: 3.186%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 4 integers unsatisfied sum - 1.16129
Cbc0038I Solution found of -61.8523
Cbc0038I Before mini branch and bound, 2664 integers at bound fixed and 0 continuous
Cbc0038I Full problem 2033

In [17]:
for v in model.get_variables():
    v.set_value(0)

In [18]:
with open('solution_sp.txt', 'r') as f:
    for line in f:
        if 'objective value' in line:
            continue
        words = line.split()
        var = model.get_variable(words[1])
        var.set_value(float(words[2]))

In [19]:
picks = []
for p in players:
    if squad[p].get_value() > 0.5:
        lp = df.loc[p]
        is_captain = 1 if captain[p].get_value() > 0.5 else 0
        is_lineup = 1 if lineup[p].get_value() > 0.5 else 0
        is_vice = 1 if vicecap[p].get_value() > 0.5 else 0
        picks.append([
            lp['Name'], lp['Pos'], lp['Team'], lp['BV'], round(lp['Pts'], 2), is_lineup, is_captain, is_vice
        ])

picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain']).sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, True])

In [20]:
picks_df

,name,pos,team,price,xP,lineup,captain,vicecaptain
8,Lindelöf,D,Man Utd,4.8,3.93,1,0,0
5,Robertson,D,Liverpool,7.1,4.57,1,0,0
6,Alexander-Arnold,D,Liverpool,7.7,4.94,1,0,0
12,Antonio,F,West Ham,6.5,4.56,1,0,0
3,Bamford,F,Leeds,6.5,4.88,1,0,0
10,Kane,F,Spurs,11.8,6.25,1,0,1
13,Mendy,G,Chelsea,5.2,3.99,1,0,0
14,Raphinha,M,Leeds,5.5,4.09,1,0,0
9,Saint-Maximin,M,Newcastle,5.1,4.56,1,0,0
7,Fernandes,M,Man Utd,11.4,6.24,1,0,0


In [28]:
picks_df.sum()

name           LindelöfRobertsonAlexander-ArnoldAntonioBamfor...
pos                                              DDDFFFGMMMMDDGM
team           Man UtdLiverpoolLiverpoolWest HamLeedsSpursChe...
price                                                        100
xP                                                         59.04
lineup                                                        11
captain                                                        1
vicecaptain                                                    1
dtype: object

In [22]:
total_xp = so.expr_sum(
    (lineup[p] + captain[p]) * df.loc[p, 'Pts'] for p in players
).get_value()
total_xp

61.547

In [23]:
price.get_value()

99.99999999999999